In [ ]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

'\n掛載Google硬碟\n安裝套件\n引用套件\nAPP應用準備\n消息準備\nhandler執行方法設計\n啟動應用\n\n'

In [ ]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''

流程解析

'''


'\n\n流程解析\n\n'

In [ ]:
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [ ]:
'''
建置主程序

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("NLuJuE98mu9aVvIIfWPrgfq9q5EoVZYP+yWK6W4O+UsYdS/GmMpUYFk7qEM2OrrDXikIV+ZRTA1V/BftphfFFqpzzA8m+ygvmWsZ7iCn0kxank+IemyqfWLa7kYok8WV5d+3cTUXJC24ndjiGG6nDgdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("b660a0127b1ab050edfe4f9d9c543b4c")

In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。

'''

from linebot.models import(
    MessageEvent,ImageMessage, TextSendMessage
)
#告訴handler當收到圖片消息的時候,執行下面的方法
@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
  #先回應圖片上傳的id
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Image has Upload'+ ' ' + event.message.id))
    #依據消息的id,向line索取檔案回來
    message_content = line_bot_api.get_message_content(event.message.id)
    #存本地端,圖片儲存
    with open(event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [ ]:
'''
影片消息，留給你
'''
from linebot.models import(
    MessageEvent,ImageMessage, TextSendMessage,VideoMessage
)
#告訴handler當收到圖片消息的時候,執行下面的方法
@handler.add(MessageEvent, message=VideoMessage)
def handle_video_message(event):
  #先回應圖片上傳的id
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Video has Upload'+ ' ' + event.message.id))
    #依據消息的id,向line索取檔案回來
    message_content = line_bot_api.get_message_content(event.message.id)
    #存本地端,圖片儲存
    with open(event.message.id+'.mp4', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [ ]:
'''
音訊消息，留給你
'''
from linebot.models import(
    MessageEvent,ImageMessage, TextSendMessage, VideoMessage, AudioMessage
)
#告訴handler當收到圖片消息的時候,執行下面的方法
@handler.add(MessageEvent, message=AudioMessage)
def handle_audiio_message(event):
  #先回應圖片上傳的id
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Audio has Upload'+ ' ' + event.message.id))
    #依據消息的id,向line索取檔案回來
    message_content = line_bot_api.get_message_content(event.message.id)
    #存本地端,圖片儲存
    with open(event.message.id+'.mp3', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)

In [ ]:
# 主程序運行
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b982-34-86-31-221.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"U5307897c9c8ff435a32a1d50e95ee58e","events":[{"type":"message","message":{"type":"image","id":"14944621415542","contentProvider":{"type":"line"}},"timestamp":1634717787910,"source":{"type":"user","userId":"Ud28913e45991f8c7531b94f85a9992a1"},"replyToken":"bd3c846ae46847ffae3fd0efcd4f2b74","mode":"active"}]}


127.0.0.1 - - [20/Oct/2021 08:16:30] "POST / HTTP/1.1" 200 -


{"destination":"U5307897c9c8ff435a32a1d50e95ee58e","events":[{"type":"message","message":{"type":"video","id":"14944629278304","duration":6638,"contentProvider":{"type":"line"}},"timestamp":1634717889803,"source":{"type":"user","userId":"Ud28913e45991f8c7531b94f85a9992a1"},"replyToken":"75c474b219bc43499346f458e69e3acf","mode":"active"}]}
{"destination":"U5307897c9c8ff435a32a1d50e95ee58e","events":[{"type":"message","message":{"type":"video","id":"14944629319668","duration":6638,"contentProvider":{"type":"line"}},"timestamp":1634717890325,"source":{"type":"user","userId":"Ud28913e45991f8c7531b94f85a9992a1"},"replyToken":"609f7068b88f42d18fe0b8d16a608e8c","mode":"active"}]}


127.0.0.1 - - [20/Oct/2021 08:18:12] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2021 08:18:13] "POST / HTTP/1.1" 200 -


{"destination":"U5307897c9c8ff435a32a1d50e95ee58e","events":[{"type":"message","message":{"type":"audio","id":"14944630522916","duration":3452,"contentProvider":{"type":"line"}},"timestamp":1634717905757,"source":{"type":"user","userId":"Ud28913e45991f8c7531b94f85a9992a1"},"replyToken":"5636c3702c7c4ecea585b5f7068dcf9f","mode":"active"}]}


127.0.0.1 - - [20/Oct/2021 08:18:28] "POST / HTTP/1.1" 200 -
